### XGBoost with ADSTuner for HPO

* Imblearn for undersampling of negative class
* ADSTuner for HPO
* tuning on more parameters
* optimize using recall

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

import ads

# to use ADSTuner
from ads.hpo.search_cv import ADSTuner
from ads.hpo.stopping_criterion import *
from ads.hpo.distributions import *

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# for undersampling the negative class
from imblearn.under_sampling import RandomUnderSampler

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import pickle

%matplotlib inline

In [2]:
# check the ADS version
print(ads.__version__)

2.5.4


In [3]:
# global constants
SEED = 1324

# number of features (without (16/03) the two indicator cols)
N_FEATURES = 12 - 2

# name of col with label
TARGET = 'SeriousDlqin2yrs'

# cols with missing values
COL1_MISSING = 'MonthlyIncome'
COL2_MISSING = 'NumberOfDependents'

# nomi delle due colonne indicator (valgono 1 laddove il dato è inputato)
IND1 = 'isna_mi'
IND2 = 'isna_nod'

ind_col = [IND1, IND2]

# 16/03 added indicators
COLS_TO_DROP = ['id', 'isna_mi', 'isna_nod']

# for undersampling to make the dataset more balanced
# ratio minority samples/majority
# with this ratio I get all the positive samples
RATIO = 1./5.

In [4]:
# full dataset, not undersampled
data_full = pd.read_csv('cs-training-nonull.csv')

# remove unneeded cols
data_full = data_full.drop(COLS_TO_DROP, axis = 1)

In [5]:
cat_cols = ['age','NumberOfTime30-59DaysPastDueNotWorse',
               'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
               'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
               'NumberOfDependents']
num_cols = ['RevolvingUtilizationOfUnsecuredLines', 'DebtRatio', 'MonthlyIncome', ]

# indicators are not touched

In [6]:
# scaling and label encoding is done on data_full. After we will do resampling
# In this way coding and scaling cover entire range of values, not only for resampled data

# we don't need any scaling (it is ensambles of trees)

In [7]:
# cat cols treatment
# Code categorical columns (only season, weather, year)

# we don't need any pre-processing for cat columns

# so for XGBoost  Nan treatment no other pre-processing is needed

In [8]:
# delete rows with anomalous age
# condition to keep
condition = (data_full['age'] >= 18) & (data_full['age'] <= 100)

data_full = data_full.loc[condition]

In [14]:
# extract X: features and y, labels
features = [c for c in data_full.columns if c != TARGET]

y_train_full = data_full[TARGET].values
x_train_full = data_full.drop(TARGET, axis = 1).values

assert x_train_full.shape[1] == N_FEATURES

In [15]:
print(f'# of samples in full dataset: {x_train_full.shape[0]}')
print(f'# of positive samples: {np.sum(y_train_full)}')
print(f'# of negative samples: {x_train_full.shape[0] - np.sum(y_train_full)}')

# of samples in full dataset: 149986
# of positive samples: 10025
# of negative samples: 139961


In [16]:
# do the undersampling of the negative class, using IMblearn
rus = RandomUnderSampler(sampling_strategy=RATIO, random_state=SEED)

x_train, y_train = rus.fit_resample(x_train_full, y_train_full)

print(f'# of samples in resampled dataset: {x_train.shape[0]}')

# check ratio of classes
print(f'# of positive samples: {np.sum(y_train)}')
print(f'# of negative samples: {x_train.shape[0] - np.sum(y_train)}')

# of samples in resampled dataset: 60150
# of positive samples: 10025
# of negative samples: 50125


The resampled dataset (x_train, y_train) will be used for training

### Train the XGBoost Classifier

In [17]:
# parameters for the HPO session with Optuna
FOLDS = 5
SEED = 4321

N_TRIALS = 100
TIME_BUDGET = 7200
STUDY_NAME = "xgb-gpu01"

# ranges
LR_LOW = 1e-3
LR_HIGH = 1e-2
DEPTH_LOW = 4
DEPTH_HIGH = 10
N_ITER_LIST = [2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000]
GAMMA_LOW = 0.1
GAMMA_HIGH = 5
SUBSAMPLE_LOW = 0.1
SUBSAMPLE_HIGH = 1.

In [18]:
#
# Here we define the strategy, the space for hyper-parameters we want to explore
#
params = {
    "n_estimators": CategoricalDistribution(N_ITER_LIST),
    "learning_rate": LogUniformDistribution(low=LR_LOW, high=LR_HIGH),
    "max_depth": IntUniformDistribution(DEPTH_LOW, DEPTH_HIGH),
    "gamma" : LogUniformDistribution(low=GAMMA_LOW, high=GAMMA_HIGH),
    "subsample" : UniformDistribution(low=SUBSAMPLE_LOW, high=SUBSAMPLE_HIGH),
    "tree_method": "gpu_hist"
}

clf = xgb.XGBClassifier()


# per lista scorer sorted(sklearn.metrics.SCORERS.keys())
tuner = ADSTuner(clf, cv=FOLDS, strategy=params, scoring="recall", study_name=STUDY_NAME, n_jobs=10, random_state=SEED)

tuner.tune(x_train, y_train, exit_criterion=[TimeBudget(TIME_BUDGET)])

[I 2022-03-16 15:07:30,109] A new study created in RDB with name: xgb-gpu01


In [ ]:
# get the status to see if completed
print(f"The tuner status is: {tuner.get_status()}")

print(f"Remaining time is: {round(tuner.time_remaining, 1)} sec.")

In [29]:
# look only at completed trials, sorted with best on top. Metric chosen is in the value col.
result_df = tuner.trials[tuner.trials["state"] == "COMPLETE"].sort_values(
    by=["value"], ascending=False
)

result_df.head(10)

,number,value,datetime_start,datetime_complete,duration,params_gamma,params_learning_rate,params_max_depth,params_n_estimators,params_subsample,...,user_attrs_metric,user_attrs_split0_test_score,user_attrs_split1_test_score,user_attrs_split2_test_score,user_attrs_split3_test_score,user_attrs_split4_test_score,user_attrs_std_fit_time,user_attrs_std_score_time,user_attrs_std_test_score,state
40,40,0.441696,2022-03-16 15:50:40.278688,2022-03-16 15:56:23.493697,0 days 00:05:43.215009,4.867601,0.003555,10,2900,0.611177,...,recall,0.441397,0.434414,0.435910,0.448379,0.448379,0.687555,0.003846,0.005932,COMPLETE
41,41,0.441496,2022-03-16 15:51:43.052523,2022-03-16 15:57:16.641345,0 days 00:05:33.588822,4.844744,0.003688,10,2900,0.585022,...,recall,0.441397,0.436409,0.435411,0.446883,0.447382,0.365662,0.005398,0.005031,COMPLETE
44,44,0.441496,2022-03-16 15:54:07.757894,2022-03-16 15:59:25.541258,0 days 00:05:17.783364,4.976931,0.003768,10,2900,0.611855,...,recall,0.441397,0.434414,0.435411,0.447382,0.448878,0.610405,0.003014,0.005938,COMPLETE
42,42,0.440898,2022-03-16 15:52:44.364767,2022-03-16 15:58:19.608948,0 days 00:05:35.244181,4.916737,0.003616,10,2900,0.617713,...,recall,0.442394,0.436409,0.433915,0.446883,0.444888,1.115432,0.001861,0.004958,COMPLETE
24,24,0.439800,2022-03-16 15:28:16.483953,2022-03-16 15:32:43.065607,0 days 00:04:26.581654,4.563597,0.005158,10,2100,0.670141,...,recall,0.440399,0.433915,0.434414,0.444888,0.445387,1.243047,0.002495,0.004921,COMPLETE
71,71,0.439501,2022-03-16 16:24:00.345973,2022-03-16 16:32:20.620609,0 days 00:08:20.274636,4.320169,0.002866,10,2700,0.662606,...,recall,0.438404,0.433416,0.434913,0.443890,0.446883,1.526334,0.002947,0.005158,COMPLETE
23,23,0.439102,2022-03-16 15:26:50.461497,2022-03-16 15:30:11.586736,0 days 00:03:21.125239,4.730048,0.003669,10,2900,0.103701,...,recall,0.446883,0.438404,0.421446,0.442893,0.445885,0.259727,0.003961,0.009307,COMPLETE
20,20,0.439002,2022-03-16 15:22:18.197466,2022-03-16 15:28:16.467276,0 days 00:05:58.269810,4.758742,0.003632,10,2700,0.664981,...,recall,0.439900,0.430424,0.430923,0.447382,0.446384,0.282691,0.001675,0.007272,COMPLETE
19,19,0.438903,2022-03-16 15:21:22.200401,2022-03-16 15:29:39.502240,0 days 00:08:17.301839,4.348594,0.002886,10,2900,0.617439,...,recall,0.438404,0.434414,0.432918,0.443392,0.445387,0.619329,0.002228,0.004866,COMPLETE
74,74,0.438903,2022-03-16 16:28:30.141041,2022-03-16 16:35:41.828237,0 days 00:07:11.687196,4.496039,0.002844,9,2700,0.660533,...,recall,0.439900,0.432918,0.431920,0.444389,0.445387,1.378407,0.004990,0.005616,COMPLETE


In [30]:
def show_tuner_results(tuner):

    # to count completed
    result_df = tuner.trials[tuner.trials["state"] == "COMPLETE"].sort_values(
        by=["value"], ascending=False
    )

    print("ADSTuner session results:")
    print(f"ADSTuner has launched {tuner.trials.shape[0]} trials")
    print(f"ADSTuner has completed {result_df.shape[0]} trials")
    print()
    print(f"The best trial is the #: {tuner.best_index}")
    print(f"Parameters for the best trial are: {tuner.best_params}")
    print(f"The metric used to optimize is: {tuner.scoring_name}")
    print(f"The best score is: {round(tuner.best_score, 4)}")
    
show_tuner_results(tuner)

ADSTuner session results:
ADSTuner has launched 99 trials
ADSTuner has completed 99 trials

The best trial is the #: 40
Parameters for the best trial are: {'gamma': 4.867601210093122, 'learning_rate': 0.00355503955960397, 'max_depth': 10, 'n_estimators': 2900, 'subsample': 0.6111768133956392, 'tree_method': 'gpu_hist'}
The metric used to optimize is: recall
The best score is: 0.4417


### Train with best params

In [ ]:
%%time

clf = xgb.XGBClassifier(**tuner.best_params)

# addestro e valuto su train e su validation set
clf.fit(x_train, y_train,
        eval_set=[(x_train, y_train)],
        eval_metric='auc', verbose=100)

print()

evals_result = clf.evals_result()

#### OK, consider that the slightly higher AUC is due to the fact here we're evaluating also on train data

In [ ]:
def plot_auc(train_hist):
    plt.figure(figsize=(9,6))
    
    plt.plot(train_hist, label='Training AUC')
    plt.title('AUC')
    plt.legend(loc='lower right')
    plt.ylabel('auc')
    plt.xlabel('n_estimator')
    plt.grid(True)
    plt.show();

In [ ]:
train_hist = evals_result['validation_0']['auc']

plot_auc(train_hist)

In [ ]:
# compute accuracy on full dataset
y_pred = clf.predict(x_train_full)

# not really needed for XGBoost
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_train_full, predictions)

print("Accuracy on train set: %.2f%%" % (accuracy * 100.0))

In [ ]:
# compute confusion matrix on full dataset
cm = confusion_matrix(y_train_full, predictions)

# plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)

disp.plot();

In [ ]:
# interpretability
# feature importance
# plot

plt.figure(figsize=(9, 6))
plt.title('Features importance')
sns.barplot(x=data_full.drop(TARGET, axis = 1).columns, y=clf.feature_importances_)
plt.xticks(rotation=90)
plt.grid()
plt.show();

In [ ]:
data_full.columns

In [ ]:
# the number of FN is rather high but using recall things get better

### Prediction on the TEST set (for submission to Kaggle)

In [ ]:
# predictions on test set
orig_test = pd.read_csv('cs-test.csv')

In [ ]:
# inpute missing values, add the two indicator columns
MONTHLY_INC_MEDIAN = 5400.
N_OF_DEP_MODE = 0

orig_test['isna_mi'] = 0
orig_test.loc[orig_test[COL1_MISSING].isna(), 'isna_mi'] = 1
orig_test.loc[orig_test[COL1_MISSING].isna(), COL1_MISSING] = MONTHLY_INC_MEDIAN

orig_test['isna_nod'] = 0
orig_test.loc[orig_test[COL2_MISSING].isna(), 'isna_nod'] = 1
orig_test.loc[orig_test[COL2_MISSING].isna(), COL2_MISSING] = N_OF_DEP_MODE

In [ ]:
ind_test = orig_test[ind_col].values

In [ ]:
orig_test = orig_test.drop(ind_col, axis = 1)

In [ ]:
orig_test.columns

In [ ]:
ID_COL_NAME = 'Unnamed: 0'
xorig_test = orig_test.drop(ID_COL_NAME, axis = 1)
xorig_test = xorig_test.drop(TARGET, axis = 1)

x_test = xorig_test.values

In [ ]:
# aggiungi qui lo scaling !!!
x_test_scaled = scaler.transform(x_test)
# riaggiunge le colonne indicatore
x_test_scaled = np.c_[x_test_scaled, ind_test]

assert x_test_scaled.shape[1] == N_FEATURES

In [ ]:
# do predictions on test set (no shuffle !)
y_pred = clf.predict_proba(x_test)

# y_pred contiene le probabilità
y_pred = y_pred[:, 1]

In [ ]:
# prepara il csv per la submission
result_dict = {"Id": orig_test[ID_COL_NAME].values,
              'Probability': y_pred}

FILE_SUB = 'submission36.csv'

# build a dataframe and save to csv
result_df = pd.DataFrame(result_dict)

result_df.to_csv(FILE_SUB, index=False, float_format='%.5f')

In [ ]:
### Save Modela and scaler

In [ ]:
# save model: uso un formato semplice: pkl
pickle.dump(clf, open("credit-scoring.pkl", "wb"))

In [ ]:
# salvo anche lo scaler
pickle.dump(scaler, open("scaler.pkl", "wb"))

### Online predictions

In [ ]:
# reload the model
loaded_model = pickle.load(open("credit-scoring.pkl", "rb"))

In [ ]:
# reload the scaler
loaded_scaler = pickle.load(open("scaler.pkl", "rb"))

In [ ]:
# prepare for online predictions
# input are given as a numpy array, with no missing fields, but we need to add the two indicator columns
x_input = np.array([[1,2,3,4,5,6,7,8,9,10],
                   [1,2,3,4,5,6,7,8,9,10],
                   [1,2,3,4,5,6,7,8,9,10]])

In [ ]:
# controlli
assert x_input.shape[1] == 10
# check there are no null
assert np.sum(np.isnan(x_input)) == 0

In [ ]:
# normalize
x_input_scaled = loaded_scaler.transform(x_input)

# add two columns with 0
x_add = np.zeros((x_input.shape[0], 2))
x_input_scaled = np.c_[x_input_scaled, x_add]

In [ ]:
y_pred = loaded_model.predict(x_input_scaled)

In [ ]:
y_pred

In [ ]:
train_df[TARGET].hist();